In [1]:
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ done
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=4f2391f228f340b813f26476f91d91a0b5e9f09f6f7aac18d155f579b892ccba
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers


In [2]:
with open(file='/kaggle/input/the-jungle-book/jungleb.txt', encoding='utf-8', mode='r') as input_fp:
    text = input_fp.read()
len(text)

292391

In [3]:
from nltk.tokenize import sent_tokenize
import pandas as pd

df = pd.DataFrame(data={'text': sent_tokenize(text=text.replace('\n', ' '), language='english')})
df['text'] = df['text'].apply(func=lambda x: ' '.join(x.split()))
df = df.drop_duplicates(ignore_index=True)
df.shape

(2579, 1)

We are going to leave all the Project Gutenberg additions to the text in place.

In [4]:
from arrow import now
from sentence_transformers import SentenceTransformer
MODEL = 'all-MiniLM-L6-v2'

model_start = now()
model = SentenceTransformer(model_name_or_path=MODEL,)
embeddings = model.encode(df['text'].values.tolist())
print('model time: {}'.format(now() - model_start))

.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

Batches:   0%|          | 0/81 [00:00<?, ?it/s]

model time: 0:00:30.018412


In [5]:
from sentence_transformers import util

similarity_start = now()
cos_sim = util.cos_sim(embeddings, embeddings)

# todo use a comprehension here
all_sentence_combinations = []
for i in range(len(cos_sim)-1):
    if i > 0 and i % 200 == 0 or i == len(df) - 1:
        print('{}: {}'.format(now(), i))
    for j in range(i+1, len(cos_sim)):
        all_sentence_combinations.append([cos_sim[i][j], i, j])

print('{}: calculated similarities.'.format(now()))
all_sentence_combinations = sorted(all_sentence_combinations, key=lambda x: x[0], reverse=True)

print('similarity time: {}'.format(now() - similarity_start))

2023-12-21T17:41:05.781520+00:00: 200
2023-12-21T17:41:10.210448+00:00: 400
2023-12-21T17:41:13.368178+00:00: 600
2023-12-21T17:41:16.461353+00:00: 800
2023-12-21T17:41:19.580006+00:00: 1000
2023-12-21T17:41:21.189076+00:00: 1200
2023-12-21T17:41:22.586680+00:00: 1400
2023-12-21T17:41:25.269425+00:00: 1600
2023-12-21T17:41:26.245475+00:00: 1800
2023-12-21T17:41:27.067215+00:00: 2000
2023-12-21T17:41:27.593151+00:00: 2200
2023-12-21T17:41:27.896637+00:00: 2400
2023-12-21T17:41:27.985099+00:00: calculated similarities.
similarity time: 0:04:42.137848


In [6]:
for score, i, j in all_sentence_combinations[:10]:
    print('cosine: {:.4f} indices: {} {} text: {}  text: {}'.format(cos_sim[i][j], i, j, df['text'].values[i][:40], df['text'].values[j][:40], ))

cosine: 1.0000 indices: 1 2495 text: You may copy it, give it away or re-use   text: You may copy it, give it away or re-use 
cosine: 0.9776 indices: 1977 2106 text: All things made he--Shiva the Preserver.  text: _All things made he--Shiva the Preserver
cosine: 0.9463 indices: 2054 2057 text: Little Toomai stared again and again.  text: Little Toomai stared once more.
cosine: 0.9316 indices: 1751 1752 text: For a young cobra?  text: For a young king cobra?
cosine: 0.9099 indices: 1046 1377 text: “Ha!  text: “Hah!
cosine: 0.9082 indices: 2469 2545 text: Section 1.  text: Section 2.
cosine: 0.9006 indices: 268 429 text: “Good!  text: “Good.
cosine: 0.8983 indices: 2466 2572 text: Project Gutenberg eBooks may be modified  text: Project Gutenberg™ eBooks are often crea
cosine: 0.8972 indices: 2257 2269 text: Two Tails is a great coward.  text: None the less, Two Tails is a great cowa
cosine: 0.8959 indices: 61 133 text: “A man’s cub.  text: “I speak for the man’s cub.
